# Creating a static route in the router.
To make the exercise useful, the server should have an IP that is not reachable by the router. Create it by running:

`sudo ifconfig lo:2  1.1.1.1 netmask 255.255.255.255 up`

Test that the IP is not reachable by the router (a ping will do).

## This is the common starting code.

In [1]:
# Basic connectivity variables.
# Only change if values of lab are modified.
router = "192.168.10.4"
port = 830
username = "vagrant"
password = "vagrant"
protocol = "ssh"

In [2]:
from ydk.providers import CodecServiceProvider, NetconfServiceProvider
from ydk.services import CRUDService, CodecService
# We need to create a provider to connect with the router. 
# There is a considerable theory behind this step (netconf, restconfg, etc.), 
# but we'll leave the details aside for this practice.
provider = NetconfServiceProvider(address=router,
                                      port=port,
                                      username=username,
                                      password=password, 
                                      protocol=protocol)
crud = CRUDService()

# YDK "binds" data from/to the router into python objects. 
# Still, we might want to look at the responses in xml for a quick. The next objects allow us to do that.

cd_provider = CodecServiceProvider(type="xml")
codec = CodecService()
# In case you want verbose logging, uncomment this lines.
#import logging
#log = logging.getLogger('ydk')
#log.setLevel(logging.DEBUG)
#ch = logging.StreamHandler()
#log.addHandler(ch)

# Deleting the default static route
The router has a default route pointing to server, which makes this whole practice boring. Let us erase it first. This  is dangerously easy to do (don't do it in production, except if you know what you are doing).
We'll use the model Cisco_IOS_XR_ip_static_cfg for static routes. Check it at https://github.com/YangModels/yang/blob/master/vendor/cisco/xr/531/Cisco-IOS-XR-ip-static-cfg.yang.

In [4]:
from ydk.models.cisco_ios_xr import Cisco_IOS_XR_ip_static_cfg as xr_ip_static_cfg

In [4]:
crud.delete(provider,xr_ip_static_cfg.RouterStatic())

Now, get into the router and check that we cannot ping the IP we just created.

# Configuring a static route

A proposed code to solve this is below. Try to do it yourself first.

It is normal to not know how to tacke this problem without help. If you do not know how to configure something from scratch, one can:

- Configure directly on the router, read, and then see how the objects look like.
- Check any of the tutorials on the YDK page. They cover most of the simple cases 

To be honest, most of the code in this tasks was taken from YDK examples

If you want to configure the static route in the router and then read it, you can use the next code (although you should be able to do this by now). If you run it without configuring it, it will come up empty. To erase the route, just re-run the line in the previous section.

In [ ]:
static_routes=crud.read(provider,xr_ip_static_cfg.RouterStatic()) 
# remember this will return empty if no static route exists
print(codec.encode(cd_provider, static_routes))

In [ ]:
# Your code goes here

In [8]:
# Solution is below
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
# #
#
#
#
#
#
#
#
#
##
#
#
#
#
#
#
#
#
##
#
#
#
#
#
#
#
#
##
#
#
#
#
#
#
#
#
##
#
#
#
#
#
#
#
#
#
# Instead of the solution, try to use the xml output as guide.
temp = '''
<router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
  <default-vrf>
    <address-family>
      <vrfipv4>
        <vrf-unicast>
          <vrf-prefixes>
            <vrf-prefix>
              <prefix>1.1.1.1</prefix>
              <prefix-length>32</prefix-length>
              <vrf-route>
                <vrf-next-hop-table>
                  <vrf-next-hop-next-hop-address>
                    <next-hop-address>192.168.10.5</next-hop-address>
                  </vrf-next-hop-next-hop-address>
                </vrf-next-hop-table>
              </vrf-route>
            </vrf-prefix>
          </vrf-prefixes>
        </vrf-unicast>
      </vrfipv4>
    </address-family>
  </default-vrf>
</router-static>
'''

In [9]:
new_route_static = xr_ip_static_cfg.RouterStatic() # create object static route

vrf_unicast = new_route_static.default_vrf.address_family.vrfipv4.vrf_unicast

vrf_prefix = vrf_unicast.vrf_prefixes.VrfPrefix()
vrf_prefix.prefix = "1.1.1.1"
vrf_prefix.prefix_length = 32

vrf_next_hop_next_hop_address = vrf_prefix.vrf_route.vrf_next_hop_table.VrfNextHopNextHopAddress()
vrf_next_hop_next_hop_address.next_hop_address = "192.168.10.5"
vrf_prefix.vrf_route.vrf_next_hop_table.vrf_next_hop_next_hop_address.append(vrf_next_hop_next_hop_address)
vrf_unicast.vrf_prefixes.vrf_prefix.append(vrf_prefix)

In [10]:
# Check the data using xml.
print(codec.encode(cd_provider, new_route_static))

<router-static xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-ip-static-cfg">
  <default-vrf>
    <address-family>
      <vrfipv4>
        <vrf-unicast>
          <vrf-prefixes>
            <vrf-prefix>
              <prefix>1.1.1.1</prefix>
              <prefix-length>32</prefix-length>
              <vrf-route>
                <vrf-next-hop-table>
                  <vrf-next-hop-next-hop-address>
                    <next-hop-address>192.168.10.5</next-hop-address>
                  </vrf-next-hop-next-hop-address>
                </vrf-next-hop-table>
              </vrf-route>
            </vrf-prefix>
          </vrf-prefixes>
        </vrf-unicast>
      </vrfipv4>
    </address-family>
  </default-vrf>
</router-static>



In [11]:
crud.create(provider, new_route_static)

In [ ]:
# Check in the router that the static route exists.

In [ ]:
# After finishing this. Let us repeat the exercise but using ansible to automatite it. Check file for details.